In [1]:
import pandas as pd
import altair as alt
import streamlit as st
from tqdm import tqdm

import pandas as pd
import json
import altair as alt
import streamlit as st
from tqdm import tqdm


def init_ds(json):
    ds= {}
    keys = json.keys()
    for k in keys:
        ds[k]= []
    return ds, keys

def read_json(file):
    dataset = {}
    keys = []
    with open(file) as file_lines:
        for count, line in enumerate(file_lines):
            data = json.loads(line.strip())
            if count ==0:
                dataset, keys = init_ds(data)
            for k in keys:
                dataset[k].append(data[k])
                
        return pd.DataFrame(dataset)


In [2]:
%%time

business_df = read_json('yelp_business.json')

CPU times: user 976 ms, sys: 77.4 ms, total: 1.05 s
Wall time: 1.07 s


In [3]:
%%time

review_df = read_json('yelp_review.json')

CPU times: user 26.9 s, sys: 8.92 s, total: 35.8 s
Wall time: 40.4 s


In [4]:
%%time

user_df = read_json('yelp_user.json')

CPU times: user 15.5 s, sys: 2.68 s, total: 18.2 s
Wall time: 24.9 s


In [5]:
print(business_df.columns)
print(review_df.columns)
print(user_df.columns)

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')
Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')
Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')


In [6]:
# Merge the business and user dataframes on the 'user_id' column
business_review_df = pd.merge(business_df, review_df, on='business_id')

# Merge the business_user_df and review dataframes on the 'business_id' column
yelp_df = pd.merge(business_review_df, user_df, on='user_id')

In [7]:
# See what features we have got in the df
print(yelp_df.columns)

Index(['business_id', 'name_x', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars_x', 'review_count_x', 'is_open',
       'attributes', 'categories', 'hours', 'review_id', 'user_id', 'stars_y',
       'useful_x', 'funny_x', 'cool_x', 'text', 'date', 'name_y',
       'review_count_y', 'yelping_since', 'useful_y', 'funny_y', 'cool_y',
       'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')


In [9]:
# See what states are in the df
print(yelp_df['state'].unique())

['CA' 'MO' 'IN' 'IL' 'TN' 'PA' 'AZ' 'NJ' 'LA' 'FL' 'ID' 'DE' 'NV' 'AB'
 'HI' 'UT' 'SD' 'WA' 'CO' 'TX' 'VI' 'NC' 'MT' 'MA' 'MI' 'VT' 'XMS']


In [11]:
# There are some states that are not in the US, so we will remove them
yelp_df = yelp_df[yelp_df['state'].isin(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'])]

In [14]:
# See what are the categories for the reviews
unique_categories = yelp_df['categories'].unique()
print(unique_categories)

['Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists'
 'Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services'
 'Vegan, Coffee & Tea, Breakfast & Brunch, Vegetarian, Restaurants, Food'
 ... 'Shopping, Kitchen & Bath, Home Services, Home & Garden'
 'Movers, Home Services, Local Services, Self Storage, Junk Removal & Hauling, Packing Services'
 'Health & Medical, Supernatural Readings, Hospitals, Professional Services, Yoga, Reiki, Psychics, Arts & Entertainment, Active Life, Fitness & Instruction, Psychic Mediums, Life Coach, Home Health Care']


In [29]:
#created a list of food-related businesses (i.e. restaurants, bars, etc.)
food_categories = ['Restaurants', 'Food', 'Bars', 'Sandwiches', 'Breakfast & Brunch', 'Burgers', 'Pizza', 'American (Traditional)', 'American (New)', 'Fast Food', 'Italian', 'Mexican', 'Chinese', 'Japanese', 'Thai', 'Indian', 'Mediterranean', 'Seafood', 'Sushi Bars', 'Wine Bars', 'Pubs', 'Lounges', 'Diners', 'Steakhouses', 'Buffets', 'Chicken Wings', 'Hot Dogs', 'Bagels', 'Salad', 'Soup', 'Vegetarian', 'Vegan', 'Specialty Food', 'Food Court', 'Cocktail Bars', 'Sports Bars', 'Tapas Bars']
# Keep only the reviews that are in the food_categories list
yelp_df = yelp_df[yelp_df['categories'].isin(food_categories)]

In [44]:
#See how the df displays dates
print(yelp_df['date'].head())
#Now extract the year from the date column and create a new column called 'year'
yelp_df['year'] = pd.DatetimeIndex(yelp_df['date']).year
#Convert the year column values to integers
yelp_df['year'] = yelp_df['year'].astype(int)
#assert that the year column is an integer
assert yelp_df['year'].dtype == 'int64'
#See the first 5 rows of the year column
print(yelp_df['year'].head())
#see the span of years in the df
print(yelp_df['year'].unique())
# looks like we have data from 2005 to 2022, so we will only keep the data in the most recent 5 years
yelp_df = yelp_df[yelp_df['year'].isin([2018, 2019, 2020, 2021, 2022])]

15307    2015-11-18 00:05:48
16020    2014-02-01 06:55:23
16518    2013-03-11 01:20:22
17741    2015-11-18 03:10:59
23544    2011-07-21 02:19:14
Name: date, dtype: object
15307    2015
16020    2014
16518    2013
17741    2015
23544    2011
Name: year, dtype: int64
[2015 2014 2013 2011 2010 2008 2020 2021 2019 2017 2022 2012 2006 2007
 2016 2009 2018 2005]


In [45]:
# Nest step, we can some data cleaning to remove the rows with missing values in the data
# See how many rows have missing values
print(yelp_df.isnull().sum())
# Some are missing hours, and a few are missing attributes (have parking lot etc.), which is not a big problem for now. So we keep them


business_id             0
name_x                  0
address                 0
city                    0
state                   0
postal_code             0
latitude                0
longitude               0
stars_x                 0
review_count_x          0
is_open                 0
attributes              6
categories              0
hours                 231
review_id               0
user_id                 0
stars_y                 0
useful_x                0
funny_x                 0
cool_x                  0
text                    0
date                    0
name_y                  0
review_count_y          0
yelping_since           0
useful_y                0
funny_y                 0
cool_y                  0
elite                   0
friends                 0
fans                    0
average_stars           0
compliment_hot          0
compliment_more         0
compliment_profile      0
compliment_cute         0
compliment_list         0
compliment_note         0
compliment_p

In [46]:
# We want to add a column that shows the population density of the zipcode where the business is located
# First, we need to get the population data for each zipcode
# We will use the data from the US Census Bureau through the US Census API
# We will use the 2020 population data
# write a function to get the population data for each zipcode from the API
def get_population_data(zipcode):
    import requests
    url = 'https://api.census.gov/data/2020/pep/population?get=POP,GEONAME&for=zip%20code%20tabulation%20area:' + str(zipcode)
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None
    

In [47]:
#Now we want to add the population data to the yelp_df dataframe
#First, we need to create a new column called 'population'
yelp_df['population'] = 0
#Now we will iterate through the rows of the df and get the population data for each zipcode
for index, row in tqdm(yelp_df.iterrows()):
    zipcode = row['postal_code']
    population_data = get_population_data(zipcode)
    if population_data:
        population = population_data[1][0]
        yelp_df.loc[index, 'population'] = population
    else:
        yelp_df.loc[index, 'population'] = 0

249it [00:50,  4.95it/s]


In [48]:
# Now we also want to add the average household income for each zipcode
# We will use the data from the US Census Bureau through the US Census API
# We will use the 2020 income data
# write a function to get the income data for each zipcode from the API
def get_income_data(zipcode):
    import requests
    url = 'https://api.census.gov/data/2019/acs/acs5/profile?get=DP03_0062E&for=zip%20code%20tabulation%20area:' + str(zipcode)
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [49]:
#Now we want to add the income data to the yelp_df dataframe
#First, we need to create a new column called 'income'
yelp_df['income'] = 0
#Now we will iterate through the rows of the df and get the income data for each zipcode
for index, row in tqdm(yelp_df.iterrows()):
    zipcode = row['postal_code']
    income_data = get_income_data(zipcode)
    if income_data:
        income = income_data[1][0]
        yelp_df.loc[index, 'income'] = income
    else:
        yelp_df.loc[index, 'income'] = 0

249it [00:46,  5.32it/s]


In [61]:
# Now we want to use altair to map the business locations, the population density, and the average household income
# We only want to map the data by zipcodes, so we will group the data by zipcode
# First, we need to create a new df that only has the columns we need
yelp_df_map = yelp_df[['postal_code', 'latitude', 'longitude', 'population', 'income']]
# Now we need to group the data by zipcode
yelp_df_map = yelp_df_map.groupby(['postal_code', 'latitude', 'longitude']).mean().reset_index()
# Now we can map the data, and display it on a US map
# First, we need to import altair
import altair as alt
# Now we need to create a map of the US
# We will use the US zipcodes shapefile from the US Census Bureau
# We import it from the api for 2020 map
import requests
url = 'https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_zcta510_500k.zip'
response = requests.get(url)
with open('cb_2020_us_zcta510_500k.zip', 'wb') as f:
    f.write(response.content)
# Now use altair to read the shapefile
us_map = alt.topo_feature('cb_2020_us_zcta510_500k.zip', 'cb_2020_us_zcta510_500k')
# Now we can create a map of the US
# We will use the population density as the color of the map
# We will display the business locations on the map
with alt.data_transformers.enable('json'):
    us_map_chart = alt.Chart(us_map).mark_geoshape().encode(
        color = alt.Color('population:Q', scale=alt.Scale(scheme='yelloworangered')),
        tooltip = ['postal_code:N', 'population:Q']
    ).properties(
        width = 800,
        height = 600
    )
    us_map_chart = us_map_chart + alt.Chart(yelp_df_map).mark_circle(size=10).encode(
        longitude = 'longitude:Q',
        latitude = 'latitude:Q',
        tooltip = ['postal_code:N', 'population:Q']
    )
    us_map_chart
# We want to view it in streamlit (we have imported streamlit in the beginning)
# We will use the st.altair_chart() function to display the chart
st.altair_chart(us_map_chart)



SyntaxError: invalid syntax (3084766662.py, line 41)